In [1]:
pip install tensorflow==2.15.0 keras==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninsta

In [2]:
# Import Libraries 
import os 
import copy,glob
import pandas as pd 
import numpy as np 
import warnings
import tensorflow as tf 
import keras 


import matplotlib.pyplot as plt 
import seaborn as sns


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
import cv2

from tensorflow.keras.models import Model, load_model,Sequential
from keras.layers import Flatten,Dense
from keras.layers import Dense,GlobalAveragePooling2D,Dropout,BatchNormalization,MaxPooling2D,GlobalMaxPooling2D
# from keras.utils import np_utils
from keras .callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from keras import regularizers
from tensorflow.keras.optimizers import Adam,Adamax

from tensorflow.keras.applications import ResNet50V2, Xception
from keras.applications.mobilenet_v2 import MobileNetV2

from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, f1_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.utils import shuffle

2024-09-04 18:34:12.902021: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-04 18:34:12.902078: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-04 18:34:12.904121: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Loading Images

In [3]:
train_gen=ImageDataGenerator(rescale=1./255)
valid_gen=ImageDataGenerator(rescale=1./255,validation_split=0.4)

train_data=train_gen.flow_from_directory("/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Train",target_size=(224,224),
                                         shuffle=True,class_mode='binary',batch_size=64)
valid_data=valid_gen.flow_from_directory("/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val",target_size=(224,224),
                                         shuffle=True,class_mode='binary')

Found 2142 images belonging to 2 classes.
Found 420 images belonging to 2 classes.


# MobileNet

In [4]:
import keras 
print(keras.__version__)

import tensorflow
print(tensorflow.__version__)

2.15.0
2.15.0


# callback 

In [5]:
checkpoint_filepath = '{epoch:02d}.h5'
checkpoint_dir = os.path.dirname(checkpoint_filepath)

model_checkpoint_callback =keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1,
    save_best_only=True)

earlystop = EarlyStopping(monitor = 'val_accuracy', 
                          min_delta = 0, 
                          patience = 15,
                          verbose = 1,
                          mode='auto')

reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy',
                              factor = 0.2,
                              patience = 4,
                              verbose = 1,
                              min_delta = 0.0001)

cb = [earlystop, reduce_lr, model_checkpoint_callback]

In [6]:
base_model_mobilenet = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

base_model_mobilenet.trainable = False

model_mobilenet = Sequential([
      base_model_mobilenet,
      #GlobalAveragePooling2D(),
      Dense(256,activation='relu'),
      Dropout(0.2),
      Dense(256,activation='relu'),
      Dropout(0.2),
      Dense(1,activation='sigmoid')])

model_mobilenet.summary()

9406464/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 1280)              2257984   
 tional)                                                         
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257 

In [7]:
model_mobilenet.compile(Adam(learning_rate=.001), loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
hist_mobilenet=model_mobilenet.fit(
      train_data,
      epochs=40,
      validation_data=valid_data,
      callbacks=cb
  )

Epoch 1/40


I0000 00:00:1725474876.106197      95 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


34/34 [==============================] - ETA: 0s - loss: 0.5141 - accuracy: 0.7381
Epoch 1: val_accuracy improved from -inf to 0.63810, saving model to 01.h5
34/34 [==============================] - 35s 815ms/step - loss: 0.5141 - accuracy: 0.7381 - val_loss: 0.6685 - val_accuracy: 0.6381 - lr: 0.0010
Epoch 2/40
34/34 [==============================] - ETA: 0s - loss: 0.2794 - accuracy: 0.8814
Epoch 2: val_accuracy improved from 0.63810 to 0.66190, saving model to 02.h5
34/34 [==============================] - 8s 241ms/step - loss: 0.2794 - accuracy: 0.8814 - val_loss: 0.6797 - val_accuracy: 0.6619 - lr: 0.0010
Epoch 3/40
34/34 [==============================] - ETA: 0s - loss: 0.1859 - accuracy: 0.9183
Epoch 3: val_accuracy improved from 0.66190 to 0.70714, saving model to 03.h5
34/34 [==============================] - 8s 241ms/step - loss: 0.1859 - accuracy: 0.9183 - val_loss: 0.6509 - val_accuracy: 0.7071 - lr: 0.0010
Epoch 4/40
34/34 [==============================] - ETA: 0s - los

In [9]:
train_accuracy = hist_mobilenet.history['accuracy']
val_accuracy = hist_mobilenet.history['val_accuracy']

final_train_accuracy = train_accuracy[-1]
final_val_accuracy = val_accuracy[-1]

print(f"Final Training Accuracy: {final_train_accuracy:.4f}")
print(f"Final Validation Accuracy: {final_val_accuracy:.4f}")

Final Training Accuracy: 0.9949
Final Validation Accuracy: 0.6857


# RestNet50v2

In [10]:
# Load the ResNet50V2 model with pre-trained weights
base_model = ResNet50V2(weights='imagenet', 
                        include_top=False, 
                        input_shape=(224, 224, 3))

# Freeze the base model layers (optional, depending on whether you want to fine-tune)
base_model.trainable = False

# Build the model
model_resnet = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dense(164, activation='relu'),
    BatchNormalization(),
    Dense(1, activation='sigmoid')  # Binary classification (monkeypox vs. non-monkeypox)
])

# Compile the model
model_resnet.compile(optimizer='rmsprop',
                     loss='binary_crossentropy', 
                     metrics=['accuracy'])


cb_resnet = [EarlyStopping(patience=5,monitor='val_accuracy',mode='max',restore_best_weights=True),
              ModelCheckpoint("ResNet50.h5",save_best_only=True)]

# Train the model
hist_resnet = model_resnet.fit(
    train_data,
    validation_data=valid_data,
    epochs=10,
    callbacks=cb_resnet
)

94668760/94668760 [==============================] - 0s 0us/step
Epoch 1/10
34/34 [==============================] - ETA: 0s - loss: 0.2516 - accuracy: 0.8950

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


34/34 [==============================] - 24s 457ms/step - loss: 0.2516 - accuracy: 0.8950 - val_loss: 0.6200 - val_accuracy: 0.6381
Epoch 2/10
34/34 [==============================] - 8s 231ms/step - loss: 0.0625 - accuracy: 0.9818 - val_loss: 0.6076 - val_accuracy: 0.6952
Epoch 3/10
34/34 [==============================] - 7s 213ms/step - loss: 0.0417 - accuracy: 0.9855 - val_loss: 0.6095 - val_accuracy: 0.7262
Epoch 4/10
34/34 [==============================] - 7s 210ms/step - loss: 0.0309 - accuracy: 0.9911 - val_loss: 0.6849 - val_accuracy: 0.7071
Epoch 5/10
34/34 [==============================] - 7s 212ms/step - loss: 0.0142 - accuracy: 0.9963 - val_loss: 0.8424 - val_accuracy: 0.7333
Epoch 6/10
34/34 [==============================] - 7s 213ms/step - loss: 0.0102 - accuracy: 0.9981 - val_loss: 0.8766 - val_accuracy: 0.7595
Epoch 7/10
34/34 [==============================] - 7s 207ms/step - loss: 0.0117 - accuracy: 0.9972 - val_loss: 1.0318 - val_accuracy: 0.6786
Epoch 8/10
34/34

In [11]:
# Print the training and validation accuracy
train_accuracy = hist_resnet.history['accuracy']
val_accuracy = hist_resnet.history['val_accuracy']

# Print the final accuracy
final_train_accuracy = train_accuracy[-1]
final_val_accuracy = val_accuracy[-1]

print(f"Final Training Accuracy: {final_train_accuracy:.4f}")
print(f"Final Validation Accuracy: {final_val_accuracy:.4f}")

Final Training Accuracy: 0.9977
Final Validation Accuracy: 0.6952


# Xception

In [12]:
base_model_xception=Xception(include_top=False,
                            input_shape=(224,224,3))

base_model_xception.trainable=False 

model_xception=Sequential([base_model_xception,
                          GlobalAveragePooling2D(),
                          Dense(1024,activation='relu'),
                          BatchNormalization(),
                          Dense(1,activation='sigmoid')])
model_xception.summary()

83683744/83683744 [==============================] - 0s 0us/step
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d_2  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_6 (Dense)             (None, 1024)              2098176   
                                                                 
 batch_normalization_6 (Bat  (None, 1024)              4096      
 chNormalization)                                                
                                                                 
 dense_7 (Dense)             (None, 1)                 1025      
                                                       

In [13]:
model_xception.compile(loss = 'binary_crossentropy', optimizer ='sgd', metrics= ['accuracy'])
     

In [14]:
hist_xception=model_xception.fit(train_data, epochs = 20 , validation_data = valid_data,  callbacks=cb)
     

Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 0.4667 - accuracy: 0.7941
Epoch 1: val_accuracy did not improve from 0.71667
34/34 [==============================] - 29s 582ms/step - loss: 0.4667 - accuracy: 0.7941 - val_loss: 0.6757 - val_accuracy: 0.5929 - lr: 0.0100
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 0.2284 - accuracy: 0.9286
Epoch 2: val_accuracy did not improve from 0.71667
34/34 [==============================] - 11s 329ms/step - loss: 0.2284 - accuracy: 0.9286 - val_loss: 0.6634 - val_accuracy: 0.6238 - lr: 0.0100
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 0.1709 - accuracy: 0.9514
Epoch 3: val_accuracy did not improve from 0.71667
34/34 [==============================] - 11s 323ms/step - loss: 0.1709 - accuracy: 0.9514 - val_loss: 0.6639 - val_accuracy: 0.6000 - lr: 0.0100
Epoch 4/20
34/34 [==============================] - ETA: 0s - loss: 0.1357 - accuracy: 0.9636
Epoch 4: val_accuracy did not improv

In [15]:
train_accuracy = hist_xception.history['accuracy']
val_accuracy = hist_xception.history['val_accuracy']

final_train_accuracy = train_accuracy[-1]
final_val_accuracy = val_accuracy[-1]

print(f"Final Training Accuracy: {final_train_accuracy:.4f}")
print(f"Final Validation Accuracy: {final_val_accuracy:.4f}")

Final Training Accuracy: 0.9879
Final Validation Accuracy: 0.5929
